In [10]:
from collections import defaultdict
from itertools import product

import pandas as pd
import torch

import roach
from relbench.data import TaskType
from relbench.datasets import get_dataset_names
from relbench.tasks import get_task_names, get_task

In [2]:
all_stores = roach.scan("relbench/2024-07-01")

In [3]:
len(all_stores)

77

In [11]:
all_stores[-1]["__roach__"]

{'project': 'relbench/2024-07-01',
 'timestamp': 1719876577141798648,
 'caller_file': '/lfs/ampere2/0/ranjanr/relbench-expts/examples/link_baseline.py',
 'done': True}

In [5]:
def wrap(name):
    return r"\texttt{" + name + r"}"

In [15]:
txt = {
    "val": "Val",
    "test": "Test",
}

# classification

In [8]:
metric = "roc_auc"
higher_is_better = True

In [13]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value not in [
            TaskType.BINARY_CLASSIFICATION.value,
            TaskType.MULTICLASS_CLASSIFICATION.value,
        ]:
            continue
        for script in [
            "gnn_node",
            "lightgbm_baseline",
        ]:
            for split in [
                "val",
                "test",
            ]:
                vals = []
                for seed in range(5):
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"].endswith(f"/{script}.py")
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task
                            and store["args"]["seed"] == seed,
                            all_stores,
                        )
                    )
                    try:
                        store = stores[-1]
                        val = store[split][metric]
                        vals.append(val)
                    except IndexError:
                        # val = float("nan")
                        # vals.append(val)
                        pass
                val = torch.tensor(vals)
                mean = val.mean().item()
                std = val.std().item()
                record = {
                    "dataset": dataset,
                    "task": task,
                    "script": script,
                    "split": split,
                    "mean": mean,
                    "std": std,
                }
                table_data.append(record)

/tmp/user/23092/ipykernel_2234070/1709334858.py:39: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std = val.std().item()


In [16]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    tex_val = (
        r"$"
        + opt_bm_open
        + f"{mean * 100:.2f}"
        + opt_bm_close
        + r"_{\pm "
        + f"{std * 100:.2f}"
        + r"}$"
    )

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val
tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_node  \
\texttt{rel-amazon} \texttt{user-churn}      Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
                    \texttt{item-churn}      Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{user-visits}     Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-clicks}     Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-event}  \texttt{user-repeat}     Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-ignore}     Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-f1}     \texttt{driver-dnf}      Val   $\bm{71.64}_{\pm nan}$   
                                             Test  $\bm{74.59}_{\pm nan}$   
                    \texttt{driver-top3}     Val   $\bm{63.02}_{\pm nan}$   
                                             Test  $\bm{68.73}_{\pm nan}$   
\texttt{rel-hm}     \texttt{user-churn}      Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-stack}  \texttt{user-engagement} Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-badge}      Val     $\bm{nan}_{\pm nan}$   
                                             Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-trial}  \texttt{study-outcome}   Val   $\bm{64.70}_{\pm nan}$   
                                             Test  $\bm{66.82}_{\pm nan}$   

                                                        lightgbm_baseline  
\texttt{rel-amazon} \texttt{user-churn}      Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
                    \texttt{item-churn}      Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
\texttt{rel-avito}  \texttt{user-visits}     Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
                    \texttt{user-clicks}     Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
\texttt{rel-event}  \texttt{user-repeat}     Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
                    \texttt{user-ignore}     Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
\texttt{rel-f1}     \texttt{driver-dnf}      Val   $\bm{66.99}_{\pm nan}$  
                                             Test  $\bm{57.62}_{\pm nan}$  
                    \texttt{driver-top3}     Val   $\bm{69.60}_{\pm nan}$  
                                             Test  $\bm{72.82}_{\pm nan}$  
\texttt{rel-hm}     \texttt{user-churn}      Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
\texttt{rel-stack}  \texttt{user-engagement} Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
                    \texttt{user-badge}      Val     $\bm{nan}_{\pm nan}$  
                                             Test    $\bm{nan}_{\pm nan}$  
\texttt{rel-trial}  \texttt{study-outcome}   Val   $\bm{68.20}_{\pm nan}$  
                                             Test  $\bm{70.31}_{\pm nan}$

In [18]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-5} \cmidrule{2-5}", r"\cmidrule{1-5}")
print(tex)

\begin{tabular}{lllll}
\toprule
 &  &  & gnn_node & lightgbm_baseline \\
\midrule
\multirow[c]{4}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-churn}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
\cmidrule{2-5}
 & \multirow[c]{2}{*}{\texttt{item-churn}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
\cmidrule{1-5}
\multirow[c]{4}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{\texttt{user-visits}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
\cmidrule{2-5}
 & \multirow[c]{2}{*}{\texttt{user-clicks}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
\cmidrule{1-5}
\multirow[c]{4}{*}{\texttt{rel-event}} & \multirow[c]{2}{*}{\texttt{user-repeat}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ \\
 &  & Test

# regression

In [20]:
metric = "mae"
higher_is_better = False

In [23]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value != TaskType.REGRESSION.value:
            continue
        for script in [
            "gnn_node",
            "lightgbm_baseline",
            "node_baseline",
        ]:
            for split in [
                "val",
                "test",
            ]:
                if script == "node_baseline":
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"].endswith(f"/{script}.py")
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task,
                            all_stores,
                        ),
                    )
                    for baseline in [
                        "global_zero",
                        "global_mean",
                        "global_median",
                        "entity_mean",
                        "entity_median",
                    ]:
                        try:
                            store = stores[-1]
                            val = store[baseline][split][metric]
                        except IndexError:
                            val = float("nan")
                        record = {
                            "dataset": dataset,
                            "task": task,
                            "script": baseline,
                            "split": split,
                            "mean": val,
                            "std": 0.0,
                        }
                        table_data.append(record)
                else:
                    vals = []
                    for seed in range(5):
                        stores = list(
                            filter(
                                lambda store: store["__roach__"]["caller_file"].endswith(f"/{script}.py")
                                and store["args"]["dataset"] == dataset
                                and store["args"]["task"] == task
                                and store["args"]["seed"] == seed,
                                all_stores,
                            )
                        )
                        try:
                            store = stores[-1]
                            val = store[split][metric]
                            vals.append(val)
                        except IndexError:
                            # val = float("nan")
                            # vals.append(val)
                            pass
                    val = torch.tensor(vals)
                    mean = val.mean().item()
                    std = val.std().item()
                    record = {
                        "dataset": dataset,
                        "task": task,
                        "script": script,
                        "split": split,
                        "mean": mean,
                        "std": std,
                    }
                    table_data.append(record)

/tmp/user/23092/ipykernel_2234070/846970945.py:68: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std = val.std().item()


In [24]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    if script in [
        "gnn_node",
        "lightgbm_baseline",
    ]:
        tex_val = (
            r"$"
            + opt_bm_open
            + f"{mean:.3f}"
            + opt_bm_close
            + r"_{\pm "
            + f"{std:.3f}"
            + r"}$"
        )
    else:
        tex_val = r"$" + opt_bm_open + f"{mean:.3f}" + opt_bm_close + r"$"

    rec["tex"] = tex_val

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val

tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_node  \
\texttt{rel-amazon} \texttt{user-ltv}        Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
                    \texttt{item-ltv}        Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{ads-clicks}      Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-event}  \texttt{user-attendance} Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-f1}     \texttt{driver-position} Val    $\bm{3.168}_{\pm nan}$   
                                             Test   $\bm{4.071}_{\pm nan}$   
\texttt{rel-hm}     \texttt{item-sales}      Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-stack}  \texttt{post-votes}      Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-trial}  \texttt{study-adverse}   Val   $\bm{46.416}_{\pm nan}$   
                                             Test  $\bm{44.513}_{\pm nan}$   
                    \texttt{site-success}    Val    $\bm{0.374}_{\pm nan}$   
                                             Test   $\bm{0.414}_{\pm nan}$   

                                                         lightgbm_baseline  \
\texttt{rel-amazon} \texttt{user-ltv}        Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
                    \texttt{item-ltv}        Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{ads-clicks}      Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-event}  \texttt{user-attendance} Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-f1}     \texttt{driver-position} Val    $\bm{3.476}_{\pm nan}$   
                                             Test   $\bm{4.078}_{\pm nan}$   
\texttt{rel-hm}     \texttt{item-sales}      Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-stack}  \texttt{post-votes}      Val      $\bm{nan}_{\pm nan}$   
                                             Test     $\bm{nan}_{\pm nan}$   
\texttt{rel-trial}  \texttt{study-adverse}   Val   $\bm{45.532}_{\pm nan}$   
                                             Test  $\bm{44.329}_{\pm nan}$   
                    \texttt{site-success}    Val    $\bm{0.418}_{\pm nan}$   
                                             Test   $\bm{0.424}_{\pm nan}$   

                                                  global_zero global_mean  \
\texttt{rel-amazon} \texttt{user-ltv}        Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
                    \texttt{item-ltv}        Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-avito}  \texttt{ads-clicks}      Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-event}  \texttt{user-attendance} Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-f1}     \texttt{driver-position} Val     $11.083$     $4.334$   
                                             Test    $11.926$     $4.513$   
\texttt{rel-hm}     \texttt{item-sales}      Val   $\bm{nan}$  $\bm{nan}$   
                                             Test  $\bm{nan}$  $\bm{nan}$   
\texttt{rel-stack}  \texttt{post-votes}      Val   $\bm{nan}$  $\bm{nan}$   
                      

In [25]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-10} \cmidrule{2-10}", r"\cmidrule{1-10}")
print(tex)

\begin{tabular}{llllllllll}
\toprule
 &  &  & gnn_node & lightgbm_baseline & global_zero & global_mean & global_median & entity_mean & entity_median \\
\midrule
\multirow[c]{4}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-ltv}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule{2-10}
 & \multirow[c]{2}{*}{\texttt{item-ltv}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule{1-10}
\multirow[c]{2}{*}{\texttt{rel-avito}} & \multirow[c]{2}{*}{\texttt{ads-clicks}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}$ & $\bm{nan}

# link prediction

In [29]:
metric = "link_prediction_map"
higher_is_better = True

In [30]:
table_data = []
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        task_obj = get_task(dataset, task)
        if task_obj.task_type.value != TaskType.LINK_PREDICTION.value:
            continue
        for script in [
            "gnn_link",
            "idgnn_link",
            "lightgbm_link_baseline",
            "link_baseline",
        ]:
            for split in [
                "val",
                "test",
            ]:
                if script == "link_baseline":
                    stores = list(
                        filter(
                            lambda store: store["__roach__"]["caller_file"].endswith(f"/{script}.py")
                            and store["args"]["dataset"] == dataset
                            and store["args"]["task"] == task,
                            all_stores,
                        ),
                    )
                    for baseline in [
                        "global_popularity",
                        "past_visit",
                    ]:
                        try:
                            store = stores[-1]
                            val = store[baseline][split][metric]
                        except IndexError:
                            val = float("nan")
                        record = {
                            "dataset": dataset,
                            "task": task,
                            "script": baseline,
                            "split": split,
                            "mean": val,
                            "std": 0.0,
                        }
                        table_data.append(record)
                else:
                    vals = []
                    for seed in range(5):
                        stores = list(
                            filter(
                                lambda store: store["__roach__"]["caller_file"].endswith(f"/{script}.py")
                                and store["args"]["dataset"] == dataset
                                and store["args"]["task"] == task
                                and store["args"]["seed"] == seed,
                                all_stores,
                            )
                        )
                        try:
                            store = stores[-1]
                            val = store[split][metric]
                            vals.append(val)
                        except IndexError:
                            # val = float("nan")
                            # vals.append(val)
                            pass
                    val = torch.tensor(vals)
                    mean = val.mean().item()
                    std = val.std().item()
                    record = {
                        "dataset": dataset,
                        "task": task,
                        "script": script,
                        "split": split,
                        "mean": mean,
                        "std": std,
                    }
                    table_data.append(record)

/tmp/user/23092/ipykernel_2234070/63554692.py:66: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std = val.std().item()


In [31]:
tex_tab = defaultdict(dict)
for rec in table_data:
    dataset = rec["dataset"]
    task = rec["task"]
    script = rec["script"]
    split = rec["split"]
    mean = rec["mean"]
    std = rec["std"]

    is_best = True
    filter_data = filter(
        lambda r: r["dataset"] == dataset and r["task"] == task and r["split"] == split,
        table_data,
    )
    for comp_rec in filter_data:
        comp_mean = comp_rec["mean"]
        comp_std = comp_rec["std"]
        if higher_is_better:
            if mean + std < comp_mean - comp_std:
                is_best = False
        else:
            if mean - std > comp_mean + comp_std:
                is_best = False
    opt_bm_open = r"\bm{" if is_best else ""
    opt_bm_close = r"}" if is_best else ""
    if script in [
        "gnn_link",
        "idgnn_link",
        "lightgbm_link_baseline",
    ]:
        tex_val = (
            r"$"
            + opt_bm_open
            + f"{mean * 100:.2f}"
            + opt_bm_close
            + r"_{\pm "
            + f"{std * 100:.2f}"
            + r"}$"
        )
    else:
        tex_val = r"$" + opt_bm_open + f"{mean * 100:.2f}" + opt_bm_close + r"$"

    tex_tab[script][(wrap(dataset), wrap(task), txt[split])] = tex_val
tex_df = pd.DataFrame(tex_tab)
tex_df

gnn_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-rate}        Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-review}      Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-stack}  \texttt{user-post-comment}     Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
                    \texttt{post-post-related}     Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-trial}  \texttt{condition-sponsor-run} Val    $\bm{3.08}_{\pm nan}$   
                                                   Test   $\bm{2.25}_{\pm nan}$   
                    \texttt{site-sponsor-run}      Val   $\bm{15.46}_{\pm nan}$   
                                                   Test  $\bm{11.70}_{\pm nan}$   

                                                                     idgnn_link  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-rate}        Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-review}      Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-stack}  \texttt{user-post-comment}     Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
                    \texttt{post-post-related}     Val     $\bm{nan}_{\pm nan}$   
                                                   Test    $\bm{nan}_{\pm nan}$   
\texttt{rel-trial}  \texttt{condition-sponsor-run} Val   $\bm{11.54}_{\pm nan}$   
                                                   Test  $\bm{11.46}_{\pm nan}$   
                    \texttt{site-sponsor-run}      Val   $\bm{17.46}_{\pm nan}$   
                                                   Test  $\bm{19.15}_{\pm nan}$   

                                                        lightgbm_link_baseline  \
\texttt{rel-amazon} \texttt{user-item-purchase}    Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-rate}        Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
                    \texttt{user-item-review}      Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
\texttt{rel-avito}  \texttt{user-ad-visit}         Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
\texttt{rel-hm}     \texttt{user-item-purchase}    Val    $\bm{nan}_{\pm nan}$   
                                                   Test   $\bm{nan}_{\pm nan}$   
\texttt{rel-s

In [32]:
tex = tex_df.to_latex()
tex = tex.replace(r"\multirow[t]", r"\multirow[c]")
tex = tex.replace(r"\cline", r"\cmidrule")
tex = tex.replace(r"\cmidrule{1-7} \cmidrule{2-7}", r"\cmidrule{1-7}")
print(tex)

\begin{tabular}{llllllll}
\toprule
 &  &  & gnn_link & idgnn_link & lightgbm_link_baseline & global_popularity & past_visit \\
\midrule
\multirow[c]{6}{*}{\texttt{rel-amazon}} & \multirow[c]{2}{*}{\texttt{user-item-purchase}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule{2-8}
 & \multirow[c]{2}{*}{\texttt{user-item-rate}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule{2-8}
 & \multirow[c]{2}{*}{\texttt{user-item-review}} & Val & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
 &  & Test & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}_{\pm nan}$ & $\bm{nan}$ & $\bm{nan}$ \\
\cmidrule